# Load dataset

In [ ]:
# from google.colab import drive
import datasets

# drive.mount('/content/drive')

# !unzip "/content/drive/MyDrive/AI/datasets/single character/dataset.zip" -d "/content"
# data = pd.read_csv("/content/drive/MyDrive/AI/datasets/single character/dataset.csv")
# labels = pd.read_csv("/content/drive/MyDrive/AI/datasets/single character/labels.csv")

!unzip -qq dataset.zip
ds = datasets.load_from_disk('dataset')

/users/eleves-b/2024/duc-anh.nguyen/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Resize images

In [2]:
import skimage.transform as sk
import numpy as np

new_shape = (40, 40)
def scale_input(data):
    input, output = data['input'], data['label']
    input = sk.resize(np.array(input), new_shape, mode='reflect', anti_aliasing=True)
    return {'input': input, 'label': output}

train = [scale_input(ds['train'][i]) for i in range(len(ds['train']))]
test = [scale_input(ds['test'][i]) for i in range(len(ds['test']))]

# Train

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_channels = 1  # 1 for grayscale, 3 for RGB
image_size = 40
hidden_size = 512
output_size = 229
learning_rate = 0.001
batch_size = 64
num_epochs = 10

# Define the CNN model
class CNN(nn.Module):
    def __init__(self, input_channels, hidden_size, output_size):
        super(CNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, stride=1, padding=1)  # (40x40) -> (40x40)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # (40x40) -> (20x20)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # (20x20) -> (20x20)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # (20x20) -> (10x10)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)  # (20x20) -> (20x20)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # (10x10) -> (5x5)

        # Fully connected layers
        self.fc1 = nn.Linear(32*10*10, hidden_size)  # Flattened size
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))  # Conv1 -> ReLU -> Pool
        x = self.pool2(torch.relu(self.conv2(x)))  # Conv2 -> ReLU -> Pool
        x = self.pool3(torch.relu(self.conv3(x)))  # Conv3 -> ReLU -> Pool
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))  # Fully connected
        x = self.fc2(x)  # Output layer
        return x

# Initialize model
model = CNN(input_channels, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Prepare dataset (reshape to 40x40 for CNN)
X_train = torch.stack([torch.tensor(data['input'], dtype=torch.float32).reshape(1, image_size, image_size) for data in train]).to(device)
y_train = torch.tensor([data['label'] for data in train], dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

X_test = torch.stack([torch.tensor(data['input'], dtype=torch.float32).reshape(1, image_size, image_size) for data in test]).to(device)
y_test = torch.tensor([data['label'] for data in test], dtype=torch.long).to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Validation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = len(test)
        sum_loss = 0

        for i in range(total):
            output = model(X_test[i].unsqueeze(0))  # Add batch dimension
            predicted = torch.argmax(output, dim=1).item()

            if predicted == y_test[i].item():
                correct += 1

            sum_loss += criterion(output, y_test[i].unsqueeze(0)).item()

        avg_loss = sum_loss / total
        accuracy = correct / total

        print(f"Epoch {epoch+1}: Average Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

Epoch [1/10], Step [1/101], Loss: 5.4314
Epoch [1/10], Step [101/101], Loss: 4.0368
Epoch 1: Average Loss: 3.1897, Accuracy: 0.2795
Epoch [2/10], Step [1/101], Loss: 2.4197
Epoch [2/10], Step [101/101], Loss: 1.6510
Epoch 2: Average Loss: 2.1990, Accuracy: 0.4105
Epoch [3/10], Step [1/101], Loss: 1.2286
Epoch [3/10], Step [101/101], Loss: 4.2433
Epoch 3: Average Loss: 1.8043, Accuracy: 0.5066
Epoch [4/10], Step [1/101], Loss: 0.9628
Epoch [4/10], Step [101/101], Loss: 2.7364
Epoch 4: Average Loss: 1.6992, Accuracy: 0.5590
Epoch [5/10], Step [1/101], Loss: 0.8819
Epoch [5/10], Step [101/101], Loss: 0.1010
Epoch 5: Average Loss: 1.6642, Accuracy: 0.5677
Epoch [6/10], Step [1/101], Loss: 0.4144
Epoch [6/10], Step [101/101], Loss: 0.0826
Epoch 6: Average Loss: 1.7418, Accuracy: 0.5895
Epoch [7/10], Step [1/101], Loss: 0.1867
Epoch [7/10], Step [101/101], Loss: 0.3528
Epoch 7: Average Loss: 1.7888, Accuracy: 0.5764
Epoch [8/10], Step [1/101], Loss: 0.3620
Epoch [8/10], Step [101/101], Loss: